In [ ]:
from helper.setup import *

def stats_full(scores):
    sum_stats = get_summary_stats(scores)
    x_bar = get_summary_stats(sum_stats['mean'])
    x_bar.index = ['x_bar']
    sum_stats = sum_stats.append(x_bar)
    
    mu = sum_stats.ix['x_bar','mean']
    sum_stats['In 95% CI?'] = (sum_stats['mean'] - mu).abs() < 1.96*sum_stats['SE']
    sum_stats.ix[-1,-1] = sum_stats.ix[0:-1,-1].mean()
    #roll average row to top
    #sum_stats = pd.concat([sum_stats.ix[['x_bar']],sum_stats.ix[0:-1]])
    #display(sum_stats)
    rs = sum_stats.ix[['x_bar']]
    rs['N'] = sample_size
    return rs

def stats_simple(scores):
    x_bars = scores.mean(axis=0)
    rs = get_summary_stats(x_bars)
    rs['N'] = sample_size
    return(rs)

def goodness(scores):
    x_bars = scores.mean(axis=0)
    return x_bars.mean()


%run -i read_matrices.py
#Randomly selects 9 batters
num_players = players.shape[0]
lineup = np.random.permutation(np.arange(num_players))[:9]

print_level = 0
gridDims = (2**14,1,1)
blockDims = (1024,1,1)
#gridDims = (1,1,1)
#blockDims = (3,1,1)
sample_size = 2**11

sim_list = ["parallel"]
sim_list.append("serial")
for simulation_method in sim_list:   
    
    print("\n\nSimulation Method = %s"%simulation_method)
    dir = "./"+simulation_method+"/"
    setter = dir+"simulation_setup.py"
    simulator = dir+"simulate.py"
        
    %run -i $setter    
    %run -i $simulator
    
    %time print(goodness(scores))
    %time dss = stats_full(scores)
    dss.index = ['full']
    ss = dss
    %time dss = stats_simple(scores)
    ss.ix['simple'] = dss.ix[0]
    display(ss)